# CO₂-Based Closed-Loop Geothermal Well-Field Optimization

This notebook implements well-field optimization for a CO₂-based closed-loop geothermal system:
- **1 center producer** (P0) at origin
- **3 injectors** (I1, I2, I3) on inner ring at fixed angles (0, 2π/3, 4π/3)
- **4 outer producers** (P1, P2, P3, P4) on outer ring with variable angles

## Optimization Variables
- `R_in`: Inner ring radius [m]
- `R_out`: Outer ring radius [m] (constraint: R_out ≥ R_in + ΔR_min)
- `θ0`: Global rotation for outer ring [rad]
- `ε1, ε2, ε3`: Angle deviations from perfect 90° increments [rad]

## Objective Function
```
J = w1*CV_inj + w2*CV_prod + w4*CV_tof - w3*(τ_years / τ_ref) + Penalty
```

Where:
- `CV_inj`: Coefficient of variation of injector pressure drops
- `CV_prod`: Coefficient of variation of producer pressure drops
- `CV_tof`: Coefficient of variation of breakthrough times
- `τ_years`: Thermal lifetime [years]
- `τ_ref`: Reference lifetime (30 years)
- `Penalty`: Quadratic penalties for constraint violations

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt

# Check for CoolProp
try:
    from CoolProp.CoolProp import PropsSI
    print('✓ CoolProp is available')
except ImportError:
    raise ImportError(
        "CoolProp is required for fluid property calculations.\n"
        "Install with: pip install CoolProp\n"
        "Or: conda install -c conda-forge coolprop"
    )

# Import wellfield package
from wellfield.config import Config, DEFAULT_CONFIG
from wellfield.geometry import (
    compute_well_coordinates,
    compute_all_well_positions,
    x_to_params,
    get_default_initial_guess,
    check_geometry_constraints,
)
from wellfield.hydraulics import (
    get_mean_fluid_properties,
    compute_pressure_field,
    compute_velocity_field,
    compute_cv_pressure,
    compute_pressure_drops,
)
from wellfield.thermal import (
    compute_reservoir_volume,
    compute_thermal_lifetime,
    compute_heat_extraction_rate,
    get_thermal_metrics,
)
from wellfield.breakthrough import compute_tof_simple, compute_streamlines
from wellfield.objective import (
    compute_objective,
    evaluate_solution,
    print_solution_summary,
)
from wellfield.optimize import run_optimization
from wellfield import plots

# Set matplotlib style
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

print('All modules imported successfully!')

## 1. Configuration

All parameters are centralized in the `Config` class. The default values exactly match the prompt specifications.

In [ ]:
# Create configuration with default values
config = DEFAULT_CONFIG

print("=" * 60)
print("CONFIGURATION PARAMETERS")
print("=" * 60)

print("\nHydraulics:")
print(f"  Total mass flow rate:  {config.M_DOT_TOTAL} kg/s")
print(f"  Permeability:          {config.K_PERM:.2e} m²")
print(f"  Reservoir thickness:   {config.H_THICK} m")
print(f"  Well diameter:         {config.D_WELL} m (radius = {config.R_WELL:.3f} m)")
print(f"  Injection pressure:    {config.P_INJ/1e6:.1f} MPa")
print(f"  Production pressure:   {config.P_PROD/1e6:.1f} MPa")
print(f"  Injection temperature: {config.T_INJ_C}°C")
print(f"  Production temperature:{config.T_PROD_C}°C")

print("\nThermal:")
print(f"  Porosity:              {config.POROSITY}")
print(f"  Rock density:          {config.RHO_ROCK} kg/m³")
print(f"  Rock heat capacity:    {config.CP_ROCK} J/(kg·K)")
print(f"  Reservoir temperature: {config.T_RES_C}°C")

print("\nGeometry Constraints:")
print(f"  Minimum well spacing:  {config.S_MIN} m")
print(f"  Minimum radial gap:    {config.DELTA_R_MIN} m")
print(f"  Min angle increment:   {config.DELTA_THETA_MIN_DEG}°")
print(f"  Max epsilon deviation: ±{config.EPS_MAX_DEG}°")

print("\nOptimization Weights:")
print(f"  w1 (CV_inj):          {config.W1}")
print(f"  w2 (CV_prod):         {config.W2}")
print(f"  w3 (τ/τ_ref):         {config.W3}")
print(f"  w4 (CV_tof):          {config.W4}")
print(f"  τ_ref:                {config.TAU_REF} years")

## 2. Fluid Properties from CoolProp

CO₂ properties (viscosity μ, density ρ, heat capacity cp) are computed at mean operating conditions using CoolProp.

In [ ]:
# Get CO2 properties at mean operating conditions
props = get_mean_fluid_properties(config)

print("CO₂ Properties at Mean Conditions")
print(f"  P_mean = {config.P_MEAN/1e6:.2f} MPa")
print(f"  T_mean = {config.T_MEAN_K:.2f} K ({config.T_MEAN_K - 273.15:.1f}°C)")
print()
print(f"  Viscosity μ   = {props['mu']:.4e} Pa·s")
print(f"  Density ρ     = {props['rho']:.2f} kg/m³")
print(f"  Heat capacity = {props['cp']:.2f} J/(kg·K)")

## 3. Initial Layout

Visualize the initial well configuration before optimization.

In [ ]:
# Get initial guess for optimization
x0 = get_default_initial_guess(config)
R_in_init, R_out_init, theta0_init, eps1_init, eps2_init, eps3_init = x_to_params(x0)

print("Initial Configuration:")
print(f"  R_in  = {R_in_init:.0f} m")
print(f"  R_out = {R_out_init:.0f} m")
print(f"  θ0    = {np.rad2deg(theta0_init):.1f}°")
print(f"  ε1    = {np.rad2deg(eps1_init):.1f}°")
print(f"  ε2    = {np.rad2deg(eps2_init):.1f}°")
print(f"  ε3    = {np.rad2deg(eps3_init):.1f}°")

# Compute initial well coordinates
coords_init = compute_well_coordinates(R_in_init, R_out_init, theta0_init, eps1_init, eps2_init, eps3_init)

# Plot initial layout
fig = plots.plot_well_layout(coords_init, config)
plt.title('Initial Well Layout', fontsize=14, fontweight='bold')
plt.show()

## 4. Initial Solution Evaluation

Evaluate the initial configuration before optimization.

In [ ]:
# Evaluate initial solution
metrics_init = evaluate_solution(x0, config)

print("Initial Solution Metrics:")
print(f"  Objective J = {metrics_init['J']:.4f}")
print(f"  CV_inj      = {metrics_init['CV_inj']:.4f}")
print(f"  CV_prod     = {metrics_init['CV_prod']:.4f}")
print(f"  CV_tof      = {metrics_init['CV_tof']:.4f}")
print(f"  τ_years     = {metrics_init['tau_years']:.1f} years")
print(f"  Penalty     = {metrics_init['penalty']:.2e}")

# Check constraints
is_valid, violations, geo_metrics = check_geometry_constraints(
    R_in_init, R_out_init, eps1_init, eps2_init, eps3_init, config
)
print(f"\nConstraints: {'All satisfied ✓' if is_valid else 'VIOLATED ✗'}")
if violations:
    for v in violations:
        print(f"  - {v}")

## 5. Run Optimization

Using `scipy.differential_evolution` with fixed random seed for reproducibility.

In [ ]:
# Run the optimizer
x_best, opt_info = run_optimization(config, verbose=True)

## 6. Optimization Results Summary

Print detailed summary of the optimized solution.

In [ ]:
# Print detailed summary
print_solution_summary(x_best, config)

In [ ]:
# Get optimized parameters and coordinates
R_in_opt, R_out_opt, theta0_opt, eps1_opt, eps2_opt, eps3_opt = x_to_params(x_best)
coords_opt = compute_well_coordinates(R_in_opt, R_out_opt, theta0_opt, eps1_opt, eps2_opt, eps3_opt)
all_pos, inj_pos, prod_pos = compute_all_well_positions(R_in_opt, R_out_opt, theta0_opt, eps1_opt, eps2_opt, eps3_opt)

# Get full metrics
metrics_opt = evaluate_solution(x_best, config)

print("\nOptimized Variables:")
print(f"  R_in  = {R_in_opt:.1f} m")
print(f"  R_out = {R_out_opt:.1f} m")
print(f"  θ0    = {np.rad2deg(theta0_opt):.1f}°")
print(f"  ε1    = {np.rad2deg(eps1_opt):.2f}°")
print(f"  ε2    = {np.rad2deg(eps2_opt):.2f}°")
print(f"  ε3    = {np.rad2deg(eps3_opt):.2f}°")
print(f"  ε4    = {np.rad2deg(metrics_opt['eps4']):.2f}° (derived)")

## 7. Required Plots

### 7.1 Well Layout Plot

In [ ]:
# Plot 1: Well layout with labels P0, I1..I3, P1..P4
fig = plots.plot_well_layout(coords_opt, config)
plt.title('Optimized Well Layout: 1 Center + 3 Injectors + 4 Outer Producers', fontsize=14, fontweight='bold')
plt.show()

### 7.2 Pressure Map

In [ ]:
# Plot 2: Pressure contour map
fig = plots.plot_pressure_map(coords_opt, config)
plt.show()

### 7.3 Streamlines with Seeds and Capture Circles

In [ ]:
# Plot 3: Streamlines + seeds + capture circles
fig = plots.plot_streamlines(coords_opt, config)
plt.show()

### 7.4 Pressure Drops and Breakthrough Times

In [ ]:
# Plot 4: Bar plots for Δp and t_bt
fig = plots.plot_metrics_bars(metrics_opt)
plt.show()

### 7.5 Optimization Convergence

In [ ]:
# Plot 5: Convergence history
fig = plots.plot_convergence(opt_info['history'])
plt.show()

### 7.6 Summary Figure

In [ ]:
# Summary figure with all plots
fig = plots.create_summary_figure(x_best, metrics_opt, config, figsize=(16, 12))
plt.show()

## 8. Thermal Model Details

Reservoir volume uses a conical frustum model:
```
V_res = (1/3) π H (R_out² + R_out·R_in + R_in²)
```

In [ ]:
# Thermal metrics
thermal = get_thermal_metrics(R_in_opt, R_out_opt, config)

print("Thermal Model Results:")
print("=" * 50)
print(f"\nReservoir Volume (frustum):")
print(f"  V_res = (1/3)π·H·(R_out² + R_out·R_in + R_in²)")
print(f"  V_res = {thermal['V_res']:.2e} m³ = {thermal['V_res']/1e9:.2f} km³")
print(f"\nHeat Content:")
print(f"  Rock mass:      {thermal['m_rock']:.2e} kg")
print(f"  Water mass:     {thermal['m_water']:.2e} kg")
print(f"  Q_res = {thermal['Q_res_GJ']:.1f} GJ")
print(f"\nHeat Extraction Rate:")
print(f"  Q̇_CO₂ = ṁ·cp·(T_prod - T_inj)")
print(f"  Q̇_CO₂ = {thermal['Q_dot_MW']:.2f} MW")
print(f"\nThermal Lifetime:")
print(f"  τ = Q_res / (Q̇_CO₂ · 3600 · 24 · 365)")
print(f"  τ = {thermal['tau_years']:.1f} years")

## 9. Comparison: Initial vs Optimized

In [ ]:
# Compare initial and optimized solutions
print("Comparison: Initial vs Optimized")
print("=" * 60)
print(f"{'Metric':<20} {'Initial':>15} {'Optimized':>15} {'Change':>12}")
print("-" * 60)
print(f"{'J (objective)':<20} {metrics_init['J']:>15.4f} {metrics_opt['J']:>15.4f} {(metrics_opt['J']-metrics_init['J'])/abs(metrics_init['J'])*100:>+11.1f}%")
print(f"{'CV_inj':<20} {metrics_init['CV_inj']:>15.4f} {metrics_opt['CV_inj']:>15.4f} {(metrics_opt['CV_inj']-metrics_init['CV_inj'])/max(metrics_init['CV_inj'], 1e-10)*100:>+11.1f}%")
print(f"{'CV_prod':<20} {metrics_init['CV_prod']:>15.4f} {metrics_opt['CV_prod']:>15.4f} {(metrics_opt['CV_prod']-metrics_init['CV_prod'])/max(metrics_init['CV_prod'], 1e-10)*100:>+11.1f}%")
print(f"{'CV_tof':<20} {metrics_init['CV_tof']:>15.4f} {metrics_opt['CV_tof']:>15.4f} {(metrics_opt['CV_tof']-metrics_init['CV_tof'])/max(metrics_init['CV_tof'], 1e-10)*100:>+11.1f}%")
print(f"{'τ_years':<20} {metrics_init['tau_years']:>15.1f} {metrics_opt['tau_years']:>15.1f} {(metrics_opt['tau_years']-metrics_init['tau_years'])/metrics_init['tau_years']*100:>+11.1f}%")
print(f"{'R_in [m]':<20} {R_in_init:>15.0f} {R_in_opt:>15.0f}")
print(f"{'R_out [m]':<20} {R_out_init:>15.0f} {R_out_opt:>15.0f}")
print("=" * 60)

## 10. Final Constraint Verification

In [ ]:
# Verify all constraints are satisfied
from wellfield.geometry import compute_minimum_spacing

is_valid, violations, geo_metrics = check_geometry_constraints(
    R_in_opt, R_out_opt, eps1_opt, eps2_opt, eps3_opt, config
)

print("Final Constraint Verification")
print("=" * 60)
print(f"\n1. Radial gap: R_out - R_in = {R_out_opt - R_in_opt:.1f} m >= {config.DELTA_R_MIN} m")
print(f"   Status: {'✓ OK' if R_out_opt - R_in_opt >= config.DELTA_R_MIN else '✗ VIOLATED'}")

min_spacing = compute_minimum_spacing(all_pos)
print(f"\n2. Minimum well spacing: {min_spacing:.1f} m >= {config.S_MIN} m")
print(f"   Status: {'✓ OK' if min_spacing >= config.S_MIN else '✗ VIOLATED'}")

eps4 = -(eps1_opt + eps2_opt + eps3_opt)
print(f"\n3. Angle increments (must be >= {config.DELTA_THETA_MIN_DEG}°):")
for k, eps in enumerate([eps1_opt, eps2_opt, eps3_opt, eps4], 1):
    inc_deg = np.rad2deg(np.pi/2 + eps)
    status = '✓ OK' if inc_deg >= config.DELTA_THETA_MIN_DEG else '✗ VIOLATED'
    print(f"   Increment {k}: {inc_deg:.1f}° {status}")

print(f"\n{'='*60}")
print(f"Overall: {'All constraints satisfied ✓' if is_valid else 'Some constraints VIOLATED ✗'}")
if violations:
    print("Violations:")
    for v in violations:
        print(f"  - {v}")